In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import plot_utils as u
import scipy.signal as signal
import re

In [ ]:
result_dir = Path("beta_modulation/")

In [ ]:
tt = np.loadtxt("../Cortex_BasalGanglia_DBS_model/burst_times_1.txt", delimiter=",")
aa = np.loadtxt("../Cortex_BasalGanglia_DBS_model/burst_level_1.txt", delimiter=",")
stt, modulation_signal = u.burst_txt_to_signal(tt, aa, 6000, 30000, 0.01)
plt.figure(figsize=(20, 10))
plt.plot(stt, modulation_signal)
plt.scatter(tt, aa)

In [ ]:
fontsize = 15
fontsize_big = 22

for d in result_dir.iterdir():
    modulation_amplitude = 100
    modulation_amplitude_text = "???"
    if (d / "Cortical_Pop").exists() and (d / "Cortical_Pop").is_dir():
        if len(list(d.glob("*.out"))) == 1:
            output_file = list(d.glob("*.out"))[0]
            with open(output_file, 'r') as f:
                for line in f:
                    m = re.match("'beta_burst_modulation_scale': ([\.0-9]+),", line)
                    if m:
                        modulation_amplitude_text = m.groups()[0]
                        modulation_amplitude *= float(modulation_amplitude_text)
        times = np.loadtxt(d / "controller_sample_times.csv")
        beta = np.loadtxt(d / "controller_beta_values.csv")
        sig = u.load_cortical_soma_voltage(d)
        spikes = u.find_population_spikes(sig)
        plt.figure(figsize=(28, 10))
        skip = 3
        for cell in spikes[::skip]:
            x, y = list(zip(*cell))
            x = [e / 2 for e in x]
            y = [e / skip for e in y]
            spikes = plt.scatter(x, y, s=5, c="#666666")
        ax = plt.gca()
        modulation, = ax.plot(stt, modulation_amplitude * modulation_signal - 1)
        controller_beta, = ax.plot(times * 1000, 8000 * beta - 7)
        zero_beta = ax.axhline(-7, color='black')
        ax.set_title(f"{d.name}, modulation amplitude={modulation_amplitude_text}", fontsize=fontsize)
        ax.set_xlabel('Time [ms]', fontsize=fontsize)
        ax.spines['top'].set_visible(False)
        ax.spines['right'].set_visible(False)
        ax.set_yticks([])
        ax.set_yticklabels([])
        plt.setp(ax.get_xticklabels(), fontsize=fontsize)
        ax.legend([spikes, modulation, controller_beta, zero_beta], ['CTX neuron spikes', 'Modulating current', 'Controller beta', 'Zero beta level'], fontsize=fontsize, bbox_to_anchor=(0, 0.05), loc="lower left")

In [ ]:
for d in result_dir.iterdir():
    times = np.loadtxt(d / "controller_sample_times.csv")
    beta = np.loadtxt(d / "controller_beta_values.csv")
    plt.figure(figsize=(20, 5))
    plt.plot(times, beta)
    plt.title(d.name)
    print(f"{d.name}: {np.mean(beta)}")

In [ ]:
psd_fig = plt.figure(figsize=(12, 10))
psd_ax = plt.gca()
for d in result_dir.iterdir():
    times = np.loadtxt(d / "controller_sample_times.csv")
    beta = np.loadtxt(d / "controller_beta_values.csv")
    lfp_t, lfp = u.load_stn_lfp(d, 30000, 6000)
    freqs, psd = signal.welch(np.transpose(lfp["signal"])[0])
    psd_ax.plot(freqs, psd)
    plt.figure(figsize=(20, 5))
    plt.plot(lfp_t, lfp["signal"])
    plt.plot(times * 1000, 10 * beta)
    plt.plot(stt, modulation_signal / 1000)
    plt.title(d.name)
psd_ax.set_xlim([0, 0.1])